In [1]:
#!pip install gseapy
#!pip install mygene

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import os
sns.set()

from datetime import datetime
pd.set_option('display.max_columns', 500)

In [3]:
import gseapy as gp
from gseapy.plot import barplot, dotplot
print(gp.__version__)

import mygene
print(mygene.__version__)

0.10.8
3.2.2


In [4]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 10:50:04


In [5]:
from gseapy.parser import Biomart
bm = Biomart()
## view validated marts
marts = bm.get_marts()
## view validated dataset
datasets = bm.get_datasets(mart='ENSEMBL_MART_ENSEMBL')
## view validated attributes
attrs = bm.get_attributes(dataset='hsapiens_gene_ensembl')
## view validated filters
filters = bm.get_filters(dataset='hsapiens_gene_ensembl')
## query results
queries = ['ENSG00000125285','ENSG00000182968'] # need to be a python list
results = bm.query(dataset='hsapiens_gene_ensembl',
                       attributes=['ensembl_gene_id', 'external_gene_name', 'entrezgene_id', 'go_id'],
                       filters={'ensemble_gene_id': queries})

In [6]:
parent = os.getcwd()
print(parent)

/Users/sussman/Documents2/2023AResearch/data/Dynamic19AllSeed1001NoBrain


In [7]:
parent_dir = os.getcwd()
print(parent_dir)

#df_pathways = pd.read_csv('PATHWAYS.csv', dtype = str)
df_pathways = pd.read_csv('KEGG_2021_Human_pathways.csv', dtype = str)
df_pathways = df_pathways.drop('Unnamed: 0', axis=1)
key_list = df_pathways['Key'].to_list()
#sub_list = df_pathways['Sub-Type'].to_list()
primary_list = df_pathways['Type'].to_list()
#color_list = df_pathways['Color'].to_list()

#zip_iterator1 = zip(key_list, sub_list)
#key_sub_type_dict = dict(zip_iterator1)


#zip_iterator2= zip(key_list, color_list)
#key_color_dict = dict(zip_iterator2)

zip_iterator3 = zip(key_list, primary_list)
key_primary_dict = dict(zip_iterator3)



display(df_pathways.head())
number_of_tissues = 19
#number_of_tissues = 19
length_threshold = 100

/Users/sussman/Documents2/2023AResearch/data/Dynamic19AllSeed1001NoBrain


Key Type
0                            Metabolic pathways    1
1         Biosynthesis of secondary metabolites    1
2  Microbial metabolism in diverse environments    1
3                             Carbon metabolism    1
4               2-Oxocarboxylic acid metabolism    1

In [8]:
source_dir = 'GTEX_intermediate_data'
os.chdir(source_dir)


samples = pd.read_csv('dtaa_sample_attributes.csv', index_col = 0)

os.chdir(parent_dir)

sample_tissues = samples['SMTSD'].value_counts()

sample_tissues = sample_tissues.iloc[[0,1,
                                      2,
                                        3,
                                        4,
                                      5,6,7,8,
                                      13,
                                      15,17,
                                      19,20,
                                      23,25,
                                      27,28,
                                      30
                                     
                                        
                                     ]]


'''
sample_tissues = sample_tissues.iloc[[0,1,2,3,4,
                                      5,6,7,8,9,
                                      10,11,12,13,
                                      14,15,16,17,
                                      18,19,20,21,
                                      22,23,24,25,
                                      26,27,28,29,
                                      30]]
'''

display(sample_tissues.head(50))
sample_tissues.to_csv('ordered_tissues.csv')

tissues = sample_tissues.index.to_list()
print(len(tissues))

order_des_tissue = sample_tissues.index.to_list()
print(type(order_des_tissue))
print(order_des_tissue)


Muscle - Skeletal                      234
Skin - Sun Exposed (Lower leg)         220
Whole Blood                            217
Adipose - Subcutaneous                 197
Thyroid                                196
Artery - Tibial                        188
Nerve - Tibial                         185
Skin - Not Sun Exposed (Suprapubic)    183
Lung                                   182
Adipose - Visceral (Omentum)           152
Heart - Atrial Appendage               148
Cells - Cultured fibroblasts           139
Testis                                 129
Breast - Mammary Tissue                125
Heart - Left Ventricle                 123
Esophagus - Mucosa                     116
Artery - Aorta                         106
Liver                                  106
Esophagus - Muscularis                  98
Name: SMTSD, dtype: int64

19
<class 'list'>
['Muscle - Skeletal', 'Skin - Sun Exposed (Lower leg)', 'Whole Blood', 'Adipose - Subcutaneous', 'Thyroid', 'Artery - Tibial', 'Nerve - Tibial', 'Skin - Not Sun Exposed (Suprapubic)', 'Lung', 'Adipose - Visceral (Omentum)', 'Heart - Atrial Appendage', 'Cells - Cultured fibroblasts', 'Testis', 'Breast - Mammary Tissue', 'Heart - Left Ventricle', 'Esophagus - Mucosa', 'Artery - Aorta', 'Liver', 'Esophagus - Muscularis']


In [9]:
mge = mygene.MyGeneInfo()

def clean_sublist(some_list):
    sublist = []
    
    for i in range(len(some_list)):
        parsed_a = some_list[i].split('.')
        sublist.append(parsed_a[0])
    ginfo = mge.querymany(sublist, scopes='ensembl.gene')
    #print(ginfo)
    a_df = pd.DataFrame(np.zeros((len(some_list), 2)))
    a_df.index = sublist
    a_df.columns = [['symbol','entrez_id']]
    #print(type(ginfo))
  
    for j in range(len(ginfo)):
        #print('len ginfo: ' +str(len(ginfo)))
        g = ginfo[j]
        #print(g)
        #print (type(g))
        try:
            entrez = g['_id']
        except KeyError:
            bb = 1
        #entrez = g['_id']
        try:
            symbol = g['symbol']
        except KeyError:
            aa = 1
        
        # symbol = g['symbol']
        #print(entrez)
        #print(symbol)
        try:
            a_df.iloc[j,0] = symbol
        except: 
            a_df.iloc[j,0] = 'error symbol'
        try: 
            a_df.iloc[j,1] = entrez
        except:
            a_df.iloc[j,1] = 'error entrez'
        
    return a_df
    #display(a_df

In [10]:
def chunk1():
    os.listdir()
    r_dir = 'Raw'
    threeWaydDict = '3WayDictionary'
    enAnalyis = 'Enrichment Analysis'
    #print(os.getcwd())
    os.chdir(r_dir)
    #print(os.getcwd())
    raw_eigen_df = pd.read_csv('eigengenes_numbered.csv', index_col=0)
    #display(raw_eigen_df.head(2))
    #print(raw_eigen_df.shape)
    eigen_cor = raw_eigen_df.corr()
    #display(eigen_cor.head(2))
    #sns.set(rc={'figure.figsize':(15,12)})
    #sns.heatmap(eigen_cor, vmin = -1, vmax=1, cmap='coolwarm')
    #plt.show()
    labels = pd.read_csv('modulesNumber.csv',index_col = 0)
    #print(labels.shape)
    #display(labels.sample(2))
    label_names_count = labels['module_number'].value_counts()
    #display(label_names_count.head(40))
    label_names_number= labels['col3'].value_counts()
    matrix_df = pd.read_csv('sk_residual_matrix_w_const.csv',index_col=0)
    #print(matrix_df.shape)
    matrix_df_T = matrix_df.T
    genes = matrix_df_T.index.tolist()
    #print(len(genes))
    #print(genes[0])
    #print(genes[1])
    labels.index = genes
    #display(labels.head())
    matrix_df_T = matrix_df_T.join(labels['module_number'])
    #display(matrix_df_T.head())
    grouped = matrix_df_T.groupby(matrix_df_T.module_number)
    #print(type(grouped))
    #print(matrix_df_T.col2)
    
    listEigenEnsmbl = []
    listEigenDF = []
    list3Way = []
    listEnrichAnalysis = []

    
    eigen_colors = label_names_count.index.tolist()
    #print('eigen_colors')
    #print(eigen_colors)
    for i in range(label_names_number.shape[0]):
        listEigenEnsmbl.append('EigengeneEnsmbl' + str(i))
    
        listEigenDF.append('Eigen_DF' + str(i))
        list3Way.append('Eigen3WayList' + eigen_colors[i])
        listEnrichAnalysis.append('EA' + eigen_colors[i])
    color_label_list = label_names_count.index.tolist()
    
    #print(listEigenEnsmbl)
    print("")
    #print(listEigenDF)
    print("")
    #print(list3Way)   
    
    #print(os.getcwd())
    #print(os.listdir())
    os.chdir('../')
    dataframes = os.chdir('DataFrames')

    for i in range(label_names_number.shape[0]):
        #display(listEigenDF[i].head(2))
        listEigenDF[i] = grouped.get_group(color_label_list[i])
        listEigenDF[i] = listEigenDF[i].drop('module_number', axis=1)
        listEigenEnsmbl[i] = listEigenDF[i].index.tolist()
        df_name = "df_" + eigen_colors[i]+'.csv'
        listEigenDF[i].to_csv(df_name)
        
    os.chdir('../')
    
    for i in range(label_names_number.shape[0]):
        #print("Eigen: "+ str(i))
        some_df = clean_sublist(listEigenEnsmbl[i])
        #display(some_df.head(1))
        #print(os.getcwd())
        os.chdir(threeWaydDict)
        some_df.to_csv(list3Way[i]+'.csv')
        os.chdir('../')
        #os.chdir(parent)
    
        symbols1 = some_df.iloc[:,0].tolist()
        enr = gp.enrichr(gene_list=symbols1,
                 #gene_sets=['KEGG_2016','KEGG_2013','GO_Biological_Process_2018','huMAP'],
                 #gene_sets=['KEGG_2016','KEGG_2013'],
                 #gene_sets=['KEGG_2016'], 
                 gene_sets=['KEGG_2021_Human'],
                 organism='Human', # don't forget to set organism to the one you desired! e.g. Yeast
                 description='test_name',
                 outdir='test/enrichr_kegg',
                 # no_plot=True,
                 cutoff=0.5 # test dataset, use lower value from range(0,1)
                )
        #display(enr.results.head(1))
        os.chdir(enAnalyis)
        enr.results.to_csv(listEnrichAnalysis[i]+'.csv')
        os.chdir('../')
        #barplot(enr.res2d,title=listEnrichAnalysis[i])
        #plt.show()
        #dotplot(enr.res2d, title=listEnrichAnalysis[i],cmap='viridis_r')
        #plt.show()
    
    
        #os.chdir(parent)
        os.chdir(enAnalyis)
        enrichfilesRaw = os.listdir()
        os.chdir('../')
        #print(enrichfilesRaw)
        #print(len(enrichfilesRaw))

        enrichfiles = [item for item in enrichfilesRaw if item.startswith("EA")]

        #print(enrichfiles)
        #print(len(enrichfiles))
        
        hsa_list = []
        for i in range(len(enrichfiles)):
            os.chdir(enAnalyis)
            df = pd.read_csv(enrichfiles[i],index_col=0)
            key_term = df['Term'].loc[0]
            key_term_split = key_term
            #print(key_term)
            #print(key_term_split)
            key_hsa = key_term_split
            hsa_list.append(key_hsa)
            #display(df.head(2))
            os.chdir('../')
    

        #print(hsa_list)
        #print(len(hsa_list))

        df = pd.DataFrame(list(zip(enrichfiles, hsa_list)),
               columns =['Name', 'hsac'])

        df['Color'] = ""

        df['Subtype'] =""
        df['Type'] = ""
        df['Plurality'] = ""
        df['Module 1'] = ""
        df['Module 2'] = ""
        df['Module 3'] = ""
        df['Module 4'] = ""
        df['Module 5'] = ""
        df['Module 6'] = ""
        df['Top'] = ""
        df['Total'] = ""

        #display(df.head(12))
        '''
        try:
            zip.getinfo('something')
        except KeyError:
             print('Can not find "something"')
        '''
        
        for i in range(df.shape[0]):
            #display(df.head())
            val_key = df['hsac'].loc[i]
            #print(val_key)
            val_key = val_key
            #print(val_key)
            #try:
            #    df['Color'].loc[i] = key_color_dict[val_key]
            #except KeyError:
            #    print('\t KEY ERROR')
            #    break
            #df['Subtype'].loc[i] = key_sub_type_dict[val_key]
            #df['Type'].loc[i] = key_primary_dict[val_key]
            #print(key_sub_type_dict[val_key])
            #print(key_color_dict[val_key])
        
        #print(os.getcwd())
        os.chdir(enAnalyis)
        df.to_csv('Summary.csv')
        #print(os.getcwd())
        os.chdir('../')
        #print(os.getcwd())
            
        
        
        

    
    
    #os.chdir('../')
    
    #display(label_names_number.head(40))
    
    
    

In [11]:
tissue_dir = order_des_tissue
print(len(tissue_dir))

for i in range(len(tissue_dir)):
    temp_tissue = tissue_dir[i]
    temp_tissue = temp_tissue[0:length_threshold]
    tissue_dir[i] =  temp_tissue
    
print(tissue_dir)

19
['Muscle - Skeletal', 'Skin - Sun Exposed (Lower leg)', 'Whole Blood', 'Adipose - Subcutaneous', 'Thyroid', 'Artery - Tibial', 'Nerve - Tibial', 'Skin - Not Sun Exposed (Suprapubic)', 'Lung', 'Adipose - Visceral (Omentum)', 'Heart - Atrial Appendage', 'Cells - Cultured fibroblasts', 'Testis', 'Breast - Mammary Tissue', 'Heart - Left Ventricle', 'Esophagus - Mucosa', 'Artery - Aorta', 'Liver', 'Esophagus - Muscularis']


In [12]:
os.chdir(parent)
os.chdir('Tissues')
current_dir=os.getcwd()




        
        

#print(tissue_dir)
#for i in range(number_of_tissues):
for i in range(number_of_tissues):
    print(tissue_dir[i])
    os.chdir(tissue_dir[i])
    print('pre-chunk 1')
    print(os.getcwd())
    chunk1()
    os.chdir(current_dir)
    print('post-chunk 1')
    print(os.getcwd())


Muscle - Skeletal
pre-chunk 1
/Users/sussman/Documents2/2023AResearch/data/Dynamic19AllSeed1001NoBrain/Tissues/Muscle - Skeletal


querying 1-1000...done.
querying 1001-2000...done.
querying 2001-2726...done.
Finished.
5 input query terms found no hit:
	['ENSG00000116957', 'ENSG00000228439', 'ENSG00000215271', 'ENSG00000130489', 'ENSG00000277203']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-1000...done.
querying 1001-2000...done.
querying 2001-2040...done.
Finished.
4 input query terms found no hit:
	['ENSG00000182230', 'ENSG00000112096', 'ENSG00000130723', 'ENSG00000274897']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-1000...done.
querying 1001-1523...done.
Finished.
1 input query terms found no hit:
	['ENSG00000189144']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-1000...done.
querying 1001-1368...done.
Finished.
querying 1-1000...don

2023-04-14 10:57:11,529 Warning: No enrich terms using library KEGG_2021_Human when cutoff = 0.5


querying 1-339...done.
Finished.
querying 1-271...done.
Finished.


2023-04-14 10:57:25,030 Warning: No enrich terms using library KEGG_2021_Human when cutoff = 0.5


querying 1-255...done.
Finished.
querying 1-255...done.
Finished.
1 input query terms found no hit:
	['ENSG00000168078']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-241...done.
Finished.
1 input query terms found no hit:
	['ENSG00000130723']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-163...done.
Finished.
querying 1-144...done.
Finished.
querying 1-132...done.
Finished.
querying 1-130...done.
Finished.
querying 1-115...done.
Finished.


...

querying 1-112...done.
Finished.
querying 1-89...done.
Finished.
querying 1-86...done.
Finished.
1 input query terms found no hit:
	['ENSG00000182230']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
post-chunk 1
/Users/sussman/Documents2/2023AResearch/data/Dynamic19AllSeed1001NoBrain/Tissues
Esophagus - Muscularis
pre-chunk 1
/Users/sussman/Documents2/2023AResearch/data/Dynamic19AllSeed1001NoBrain/Tissues/Esophagus - Muscularis


querying 1-1000...done.
querying 1001-1837...done.
Finished.
1 input query terms found no hit:
	['ENSG00000277203']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-1000...done.
querying 1001-1331...done.
Finished.
2 input query terms found no hit:
	['ENSG00000278566', 'ENSG00000130489']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-1000...done.
querying 1001-1291...done.
Finished.
1 input query terms found no hit:
	['ENSG00000130

2023-04-14 12:07:46,104 Warning: No enrich terms using library KEGG_2021_Human when cutoff = 0.5


querying 1-182...done.
Finished.
1 input query terms found no hit:
	['ENSG00000182584']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


2023-04-14 12:07:53,280 Warning: No enrich terms using library KEGG_2021_Human when cutoff = 0.5


querying 1-175...done.
Finished.
querying 1-174...done.
Finished.
querying 1-173...done.
Finished.
2 input query terms found no hit:
	['ENSG00000284413', 'ENSG00000189144']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
querying 1-154...done.
Finished.
post-chunk 1
/Users/sussman/Documents2/2023AResearch/data/Dynamic19AllSeed1001NoBrain/Tissues


In [13]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 12:08:22
